<a href="https://colab.research.google.com/github/nancymahmoud1/Elevvo/blob/main/Loan_Approval_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
import kagglehub

import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report


In [ ]:
# Download latest version
path = kagglehub.dataset_download("architsharma01/loan-approval-prediction-dataset")

print("Path to dataset files:", path)

100%|██████████| 80.6k/80.6k [00:00<00:00, 57.8MB/s]

Extracting files...
Path to dataset files: /root/.cache/kagglehub/datasets/architsharma01/loan-approval-prediction-dataset/versions/1


In [ ]:
# LAP = Loan Approval Prediction
LAP_file_path = '/root/.cache/kagglehub/datasets/architsharma01/loan-approval-prediction-dataset/versions/1/loan_approval_dataset.csv'
LAP_data = pd.read_csv(LAP_file_path)
LAP_data.columns

Index(['loan_id', ' no_of_dependents', ' education', ' self_employed',
       ' income_annum', ' loan_amount', ' loan_term', ' cibil_score',
       ' residential_assets_value', ' commercial_assets_value',
       ' luxury_assets_value', ' bank_asset_value', ' loan_status'],
      dtype='object')

In [ ]:
# Returns a Series showing the count of missing values per column.
LAP_data.isnull().sum()

,0
loan_id,0
no_of_dependents,0
education,0
self_employed,0
income_annum,0
loan_amount,0
loan_term,0
cibil_score,0
residential_assets_value,0
commercial_assets_value,0


In [ ]:
# Remove leading and trailing spaces from column names
LAP_data.columns = LAP_data.columns.str.strip()

In [ ]:
# Exclude ID and target
X = LAP_data.drop(columns=['loan_id', 'loan_status'])

# One-hot encode categorical features
X = pd.get_dummies(X, columns=['education', 'self_employed'], drop_first=True)

# Remove all spaces from column names
X.columns = X.columns.str.replace(' ', '')

print("Modified column names (underscores):", X.columns.tolist())

# X.describe()
X.head()

Modified column names (underscores): ['no_of_dependents', 'income_annum', 'loan_amount', 'loan_term', 'cibil_score', 'residential_assets_value', 'commercial_assets_value', 'luxury_assets_value', 'bank_asset_value', 'education_NotGraduate', 'self_employed_Yes']


,no_of_dependents,income_annum,loan_amount,loan_term,cibil_score,residential_assets_value,commercial_assets_value,luxury_assets_value,bank_asset_value,education_NotGraduate,self_employed_Yes
0,2,9600000,29900000,12,778,2400000,17600000,22700000,8000000,False,False
1,0,4100000,12200000,8,417,2700000,2200000,8800000,3300000,True,True
2,3,9100000,29700000,20,506,7100000,4500000,33300000,12800000,False,False
3,3,8200000,30700000,8,467,18200000,3300000,23300000,7900000,False,False
4,5,9800000,24200000,20,382,12400000,8200000,29400000,5000000,True,True


In [ ]:
# Encode categorical data
prepro = LabelEncoder()
LAP_data['loan_status'] = prepro.fit_transform(LAP_data['loan_status'])

y_encoded = LAP_data.loan_status
print(y_encoded)

0       0
1       1
2       1
3       1
4       1
       ..
4264    1
4265    0
4266    1
4267    0
4268    0
Name: loan_status, Length: 4269, dtype: int64


In [ ]:
# Split data (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(
    X, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded
)

print("Training set size (80%) :", X_train.shape)
print("Test set size (20%) :", X_test.shape)

Training set size (80%) : (3415, 11)
Test set size (20%) : (854, 11)


In [ ]:
# Import Logistic Regression
from sklearn.linear_model import LogisticRegression

# Train a Logistic Regression model
lr = LogisticRegression(random_state=42)
lr.fit(X_train, y_train)

# Make predictions on the test set
y_pred_lr = lr.predict(X_test)
y_proba_lr = lr.predict_proba(X_test)[:, 1]

# Evaluate the model
print("Logistic Regression Accuracy:", accuracy_score(y_test, y_pred_lr))
print("Logistic Regression ROC-AUC:", roc_auc_score(y_test, y_proba_lr))
print("\nLogistic Regression Classification Report:\n", classification_report(y_test, y_pred_lr))

Logistic Regression Accuracy: 0.7400468384074942
Logistic Regression ROC-AUC: 0.8287243532560215

Logistic Regression Classification Report:
               precision    recall  f1-score   support

           0       0.72      0.95      0.82       531
           1       0.82      0.40      0.54       323

    accuracy                           0.74       854
   macro avg       0.77      0.67      0.68       854
weighted avg       0.76      0.74      0.71       854



/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
from sklearn.tree import DecisionTreeClassifier

# Train a Decision Tree model
dt = DecisionTreeClassifier(random_state=42)
dt.fit(X_train, y_train)

# Make predictions on the test set
y_pred_dt = dt.predict(X_test)
y_proba_dt = dt.predict_proba(X_test)[:, 1]

# Evaluate the model
print("Decision Tree Accuracy:", accuracy_score(y_test, y_pred_dt))
print("Decision Tree ROC-AUC:", roc_auc_score(y_test, y_proba_dt))
print("\nDecision Tree Classification Report:\n", classification_report(y_test, y_pred_dt))

Decision Tree Accuracy: 0.9707259953161592
Decision Tree ROC-AUC: 0.9667576218712284

Decision Tree Classification Report:
               precision    recall  f1-score   support

           0       0.97      0.98      0.98       531
           1       0.97      0.95      0.96       323

    accuracy                           0.97       854
   macro avg       0.97      0.97      0.97       854
weighted avg       0.97      0.97      0.97       854



### Model Comparison: Logistic Regression vs. Decision Tree

| Model              | Accuracy                       | ROC-AUC                        |
| :----------------- | :----------------------------- | :----------------------------- |
| Logistic Regression | 0.7400468384074942 | 0.8287243532560215 |
| Decision Tree      | 0.9707259953161592 | 0.9667576218712284 |
